# 10 Minutes to Koalas.

This is a short introduction to Koalas, geared mainly for new users. This notebook shows you some key differences between Pandas and Koalas.

The original Pandas tutorial is located here:
http://pandas.pydata.org/pandas-docs/stable/10min.html

Customarily, we import Pandas as follows:

In [1]:
import pandas as pd
import numpy as np
import databricks.koalas as ks

Spark it usually loaded already into the interpreter.

In [4]:
# spark

Activating the PandasOnSpark is a simple matter of importing the following package:

In [5]:
# import pandorable_sparky
# import pyarrow
# pyarrow.__version__


From now on, Spark will behave in a way that is closer to Pandas:
 - Spark DataFrames will have a large number of extra functions that mimic the Pandas functions
 - Spark columns will mimic the beahvior of Pandas series
 - the `pyspark` package and the `spark` context object will have extra functions that mimic functions found in the `pandas` package.

## Object Creation

See the [Data Structure Intro section](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dsintro) 

Creating a Series by passing a list of values, letting pandas create a default integer index:

In [2]:
s = pd.Series([1,3,5,np.nan,6,8])

In [3]:
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

Creating a DataFrame by passing a numpy array, with a datetime index and labeled columns:

In [4]:
dates = pd.date_range('20130101', periods=6)

In [5]:
dates

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

In [6]:
pdf = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))

In [7]:
pdf

,A,B,C,D
2013-01-01,0.830620,-0.637647,0.424005,0.767639
2013-01-02,1.220761,-0.247430,0.246443,2.964931
2013-01-03,0.162569,0.197740,-0.900090,0.907806
2013-01-04,1.400141,0.334369,-0.283772,-0.724484
2013-01-05,-0.533946,-0.468093,-1.199911,0.897936
2013-01-06,-1.533739,-0.238883,-0.680672,0.026554


Now, this pandas dataframe can be converted to a Spark dataframe:

In [11]:
kdf = ks.from_pandas(pdf)

In [12]:
type(kdf)

databricks.koalas.frame.DataFrame

It looks and behaves the same as a Pandas dataframe though:

In [14]:
kdf

,A,B,C,D
2013-01-01,0.830620,-0.637647,0.424005,0.767639
2013-01-02,1.220761,-0.247430,0.246443,2.964931
2013-01-03,0.162569,0.197740,-0.900090,0.907806
2013-01-04,1.400141,0.334369,-0.283772,-0.724484
2013-01-05,-0.533946,-0.468093,-1.199911,0.897936
2013-01-06,-1.533739,-0.238883,-0.680672,0.026554


Creating a DataFrame by passing a dict of objects that can be converted to series-like.

In [16]:
kdf2 = ks.from_pandas(pd.DataFrame({'A':1.,
                   'B':pd.Timestamp('20130102'),
                   'C':pd.Series(1,index=list(range(4)),dtype='float32'),
                   'D':np.array([3]*4,dtype='int32'),
                   'E':pd.Categorical(["test","train","test","train"]),
                   'F':'foo'}))

In [17]:
kdf2

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo


Having specific [dtypes](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-dtypes) . Types that are common to both Spark and Pandas are currently supported.

In [18]:
# Currently a bug on .dtypes, this will be fixed via
# https://github.com/databricks/koalas/commit/b1e9033691cce74236382c23010c1ef4b1572f0e
# targeted 0.2.0
kdf2.dtypes

AttributeError: 'DataFrame' object has no attribute 'dtypes'

If you’re using IPython, tab completion for column names (as well as public attributes) is automatically enabled. Here’s a subset of the attributes that will be completed:

In [16]:
# df2.<TAB>

As you can see, the columns A, B, C, and D are automatically tab completed. E is there as well; the rest of the attributes have been truncated for brevity.

## Viewing Data

See the [Basics section](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics) 

See the top rows of the frame. The results may not be the same as Pandas though: unlike Pandas, the data in a Spark dataframe is not _ordered_, it has no intrinsic notion of index. When asked for the head of a dataframe, Spark will just take the requested number of rows from a partition. Do not rely on it to return specific rows, use `.loc` instead.

In [20]:
kdf.head()

,A,B,C,D
2013-01-01,0.830620,-0.637647,0.424005,0.767639
2013-01-02,1.220761,-0.247430,0.246443,2.964931
2013-01-03,0.162569,0.197740,-0.900090,0.907806
2013-01-04,1.400141,0.334369,-0.283772,-0.724484
2013-01-05,-0.533946,-0.468093,-1.199911,0.897936


For now, the tail is not supported. In the context of Spark, it may be dangerous because it can easily return too many rows, which are going to saturate the memory of the host computer.

In [24]:
#kdf.tail(3)

PandasNotImplementedError: The method `pd.DataFrame.tail()` is not implemented yet.

Display the index, columns, and the underlying numpy data.

You can also retrieve the index; the index column can be ascribed to a DataFrame, see later

In [21]:
kdf.index

0   2013-01-01
1   2013-01-02
2   2013-01-03
3   2013-01-04
4   2013-01-05
5   2013-01-06
Name: __index_level_0__, dtype: datetime64[ns]

In [22]:
kdf.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

For the same reason, values is not supported: it will usually return too much data for the user to handle.

In [23]:
#kdf.values

AttributeError: 'DataFrame' object has no attribute 'values'

Describe shows a quick statistic summary of your data

In [25]:
# This currently does not work, but will be resolved soon
kdf.describe()

PandasNotImplementedError: The method `pd.DataFrame.describe()` is not implemented yet.

Transposing your data: Transposing is not allowed for now, as it can cause the data to be all aggregated into a single computer.

In [23]:
# df.T

Sorting by an axis

In [24]:
# TODO
# df.sort_index(axis=1, ascending=False)

Sorting by value

In [26]:
kdf.sort_values(by='B')

,A,B,C,D
2013-01-01,0.830620,-0.637647,0.424005,0.767639
2013-01-05,-0.533946,-0.468093,-1.199911,0.897936
2013-01-02,1.220761,-0.247430,0.246443,2.964931
2013-01-06,-1.533739,-0.238883,-0.680672,0.026554
2013-01-03,0.162569,0.197740,-0.900090,0.907806
2013-01-04,1.400141,0.334369,-0.283772,-0.724484


## Selection

**Note:** While standard Python / Numpy expressions for selecting and setting are intuitive and come in handy for interactive work, for production code, we recommend the optimized pandas data access methods, .at, .iat, .loc, .iloc and .ix.

See the indexing documentation [Indexing and Selecting Data](http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing) and [MultiIndex / Advanced Indexing](http://pandas.pydata.org/pandas-docs/stable/advanced.html#advanced)

### Getting

Selecting a single column, which yields a Series, equivalent to df.A

In [27]:
kdf['A']

2013-01-01    0.830620
2013-01-02    1.220761
2013-01-03    0.162569
2013-01-04    1.400141
2013-01-05   -0.533946
2013-01-06   -1.533739
Name: A, dtype: float64

Selecting via [], which slices the rows.

In [ ]:
#kdf[0:3]

In [30]:
kdf['20130102':'20130104']

AttributeError: module 'pyarrow.pandas_compat' has no attribute 'make_datetimetz'

### Selection by Label

See more in [Selection by Label](Selection by Label)

For getting a cross section using a label

In [29]:
# TODO
#df.loc[dates[0]]

Selection by Label

In [31]:
kdf.loc[:,['A','B']]

,A,B
2013-01-01,0.830620,-0.637647
2013-01-02,1.220761,-0.247430
2013-01-03,0.162569,0.197740
2013-01-04,1.400141,0.334369
2013-01-05,-0.533946,-0.468093
2013-01-06,-1.533739,-0.238883


Showing label slicing, both endpoints are included

In [32]:
kdf.loc['20130102':'20130104',['A','B']]

AttributeError: module 'pyarrow.pandas_compat' has no attribute 'make_datetimetz'

Reduction in the dimensions of the returned object

In [32]:
# TODO
# df.loc['20130102',['A','B']]

For getting a scalar value

In [33]:
# TODO
#df.loc[dates[0],'A']

### Selection by Position

See more in [Selection by Position](http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-integer)

Select via the position of the passed integers: this is not supported, like any method that requires accessing a specific row by its position in a dataframe. As mentioned before, there is no ordering in a Spark dataframe.

In [34]:
#df.iloc[3]

A   -0.330632
B   -1.033881
C   -0.284101
D   -0.021730
Name: 2013-01-04 00:00:00, dtype: float64

By integer slices, acting similar to numpy/python

In [35]:
#df.iloc[3:5,0:2]

,A,B
2013-01-04,-0.330632,-1.033881
2013-01-05,0.190455,-0.485405


By lists of integer position locations, similar to the numpy/python style

In [36]:
#df.iloc[[1,2,4],[0,2]]

,A,C
2013-01-02,-0.724002,0.377346
2013-01-03,-0.831722,0.828880
2013-01-05,0.190455,0.595879


For slicing rows explicitly

In [37]:
#df.iloc[1:3,:]

,A,B,C,D
2013-01-02,-0.724002,-0.311910,0.377346,-0.449372
2013-01-03,-0.831722,-1.944146,0.828880,-0.611957


For slicing columns explicitly

In [38]:
df.iloc[:,1:3]

,B,C
2013-01-01,1.240925,0.416144
2013-01-02,-0.311910,0.377346
2013-01-03,-1.944146,0.828880
2013-01-04,-1.033881,-0.284101
2013-01-05,-0.485405,0.595879
2013-01-06,1.337154,-0.242153


For getting a value explicitly

In [39]:
#df.iloc[1,1]

-0.31191034680029817

For getting fast access to a scalar (equiv to the prior method)

In [40]:
#df.iat[1,1]

-0.31191034680029817

## Boolean Indexing

Using a single column’s values to select data.

In [41]:
df[df.A > 0]

,A,B,C,D
2013-01-01,0.786412,1.240925,0.416144,1.951986
2013-01-05,0.190455,-0.485405,0.595879,0.442893
2013-01-06,1.216215,1.337154,-0.242153,-0.204078


A where operation for getting.

In [42]:
# TODO
# df[df > 0]

Using the isin() method for filtering:

In [43]:
df2 = df.copy()

In [44]:
# TODO: this is not common in Spark
#sdf2['E'] = ['one','one', 'two','three','four','three']

In [45]:
df2

,A,B,C,D
2013-01-01,0.786412,1.240925,0.416144,1.951986
2013-01-02,-0.724002,-0.311910,0.377346,-0.449372
2013-01-03,-0.831722,-1.944146,0.828880,-0.611957
2013-01-04,-0.330632,-1.033881,-0.284101,-0.021730
2013-01-05,0.190455,-0.485405,0.595879,0.442893
2013-01-06,1.216215,1.337154,-0.242153,-0.204078


In [46]:
# TODO?
#df2[df2['E'].isin(['two','four'])]

## Setting

Setting a new column only allows using constants or data from the same dataframe. You 
cannot mix data from other dataframes, even if they have the same index in pandas. The reason 
for that restriction is that aligning dataframes in Spark is very expensive (it requires joining 
the two tables), and it is typically an operation that users should explicitly do lest they have
unexpected performance issues.

In [143]:
df2 = df.copy()

In [144]:
df2['A_squared'] = df2.A * df2.A

In [145]:
df2

,A,B,C,D,A_squared
2013-01-01,-1.383219,2.257891,1.268032,-1.817541,1.913296
2013-01-02,-0.068694,-0.102864,0.283140,1.355917,0.004719
2013-01-03,-0.216320,-0.524703,-1.323747,-0.345387,0.046795
2013-01-04,0.663908,-0.401103,2.868520,-0.986433,0.440774
2013-01-05,0.474352,1.417917,0.513173,1.827536,0.225009
2013-01-06,-0.073261,0.419502,0.754491,0.779783,0.005367


In [146]:
df['F'] = 0

AssertionError: col should be Column

Setting values by label is currently not supported.

In [50]:
# TODO?
#df.at[dates[0],'A'] = 0

Setting values by position is not supported, and may never be.

In [51]:
#df.iat[0,1] = 0

Setting by assigning with a numpy array is not supported.

In [52]:
#df.loc[:,'D'] = np.array([5] * len(df))

The result of the prior setting operations

In [147]:
df

,A,B,C,D
2013-01-01,-1.383219,2.257891,1.268032,-1.817541
2013-01-02,-0.068694,-0.102864,0.283140,1.355917
2013-01-03,-0.216320,-0.524703,-1.323747,-0.345387
2013-01-04,0.663908,-0.401103,2.868520,-0.986433
2013-01-05,0.474352,1.417917,0.513173,1.827536
2013-01-06,-0.073261,0.419502,0.754491,0.779783


A where operation with setting.

In [54]:
#sdf2 = sdf.copy()

In [55]:
# TODO
#sdf2[sdf2 > 0] = -sdf2

In [56]:
#df2

,A,B,C,D
2013-01-01,0.786412,1.240925,0.416144,1.951986
2013-01-02,-0.724002,-0.311910,0.377346,-0.449372
2013-01-03,-0.831722,-1.944146,0.828880,-0.611957
2013-01-04,-0.330632,-1.033881,-0.284101,-0.021730
2013-01-05,0.190455,-0.485405,0.595879,0.442893
2013-01-06,1.216215,1.337154,-0.242153,-0.204078


## Missing Data

pandas primarily uses the value np.nan to represent missing data. It is by default not included in computations. See the Missing Data section

Reindexing is currently limited to rested the data used for indexing.

In [57]:
#df1 = df.reindex(index=dates[0:4], columns=list(df.columns) + ['E'])

In [58]:
#df1.loc[dates[0]:dates[1],'E'] = 1

In [59]:
#df1

,A,B,C,D,F,E
2013-01-01,0.000000,0.000000,0.416144,5,NaN,1.0
2013-01-02,-0.724002,-0.311910,0.377346,5,1.0,1.0
2013-01-03,-0.831722,-1.944146,0.828880,5,2.0,NaN
2013-01-04,-0.330632,-1.033881,-0.284101,5,3.0,NaN


To drop any rows that have missing data.

In [60]:
# TODO
# df1.dropna(how='any')

,A,B,C,D,F,E
2013-01-02,-0.724002,-0.31191,0.377346,5,1.0,1.0


Filling missing data

In [61]:
# TODO
# df1.fillna(value=5)

,A,B,C,D,F,E
2013-01-01,0.000000,0.000000,0.416144,5,5.0,1.0
2013-01-02,-0.724002,-0.311910,0.377346,5,1.0,1.0
2013-01-03,-0.831722,-1.944146,0.828880,5,2.0,5.0
2013-01-04,-0.330632,-1.033881,-0.284101,5,3.0,5.0


To get the boolean mask where values are nan

In [62]:
# TODO
# pd.isnull(df1)

,A,B,C,D,F,E
2013-01-01,False,False,False,False,True,False
2013-01-02,False,False,False,False,False,False
2013-01-03,False,False,False,False,False,True
2013-01-04,False,False,False,False,False,True


## Operations

See the [Basic section on Binary Ops](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-binop)

### Stats

Operations in general exclude missing data.

Performing a descriptive statistic

In [149]:
# TODO
#df.mean()

Same operation on the other axis

In [64]:
# TODO
#df.mean(1)

Operating with objects that have different dimensionality and need alignment are generally not supported.
As mentioned, they can cause surprising performance issues so PandasOnSpark will refuse to execute them in general.


In [65]:
#s = pd.Series([1,3,5,np.nan,6,8], index=dates).shift(2)

In [152]:
#s

In [151]:
# TODO
#df.sub(df.A, axis='index')

### Apply

Applying functions to the data

In [68]:
#df.apply(np.cumsum)

,A,B,C,D,F
2013-01-01,0.000000,0.000000,0.416144,5,NaN
2013-01-02,-0.724002,-0.311910,0.793490,10,1.0
2013-01-03,-1.555725,-2.256056,1.622370,15,3.0
2013-01-04,-1.886357,-3.289937,1.338269,20,6.0
2013-01-05,-1.695901,-3.775342,1.934148,25,10.0
2013-01-06,-0.479686,-2.438188,1.691996,30,15.0


In [154]:
# TODO: mntion the UDF wrappers
# df.apply(lambda x: x.max() - x.min())

### Histogramming

See more at [Histogramming and Discretization](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-discretization)

In [156]:
# TODO: wrap series
s = spark.from_pandas(
    pd.DataFrame({"x":
                  pd.Series(np.random.randint(0, 7, size=10))})).x

In [157]:
s

0    5
1    6
2    4
3    1
4    3
5    5
6    2
7    1
8    5
9    1
Name: x, dtype: int64

In [160]:
s.value_counts()

AttributeError: Field value_counts not found, possible values are __index_level_0__, x

### String Methods

Series is equipped with a set of string processing methods in the str attribute that make it easy to operate on each element of the array, as in the code snippet below. Note that pattern-matching in str generally uses [regular expressions](https://docs.python.org/2/library/re.html) by default (and in some cases always uses them). See more at [Vectorized String Methods](http://pandas.pydata.org/pandas-docs/stable/text.html#text-string-methods).

In [73]:
# TODO: wrap series
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])

In [74]:
s.str.lower()

0       a
1       b
2       c
3    aaba
4    baca
5     NaN
6    caba
7     dog
8     cat
dtype: object

## Merge

### Concat

pandas provides various facilities for easily combining together Series, DataFrame, and Panel objects with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

See the [Merging section](http://pandas.pydata.org/pandas-docs/stable/merging.html#merging)

Concatenating pandas objects together with concat():

In [75]:
df = pd.DataFrame(np.random.randn(10, 4))

In [76]:
df

,0,1,2,3
0,1.269761,-0.188683,0.988276,-1.510233
1,0.434395,-0.491800,1.900742,0.036005
2,0.317188,-1.004712,0.212755,-0.109534
3,-0.369310,0.727877,0.255676,0.327390
4,-0.615128,-1.382070,-0.547244,0.366575
5,0.231131,1.591778,-0.244228,-1.475949
6,-0.525784,0.174802,0.586159,-1.943311
7,0.707470,-0.276887,-1.169881,-0.296685
8,-0.916886,-0.438234,-1.106999,0.412588
9,0.552813,0.717973,-0.174670,1.024134


In [77]:
# break it into pieces
pieces = [df[:3], df[3:7], df[7:]]

In [78]:
pd.concat(pieces)

,0,1,2,3
0,1.269761,-0.188683,0.988276,-1.510233
1,0.434395,-0.491800,1.900742,0.036005
2,0.317188,-1.004712,0.212755,-0.109534
3,-0.369310,0.727877,0.255676,0.327390
4,-0.615128,-1.382070,-0.547244,0.366575
5,0.231131,1.591778,-0.244228,-1.475949
6,-0.525784,0.174802,0.586159,-1.943311
7,0.707470,-0.276887,-1.169881,-0.296685
8,-0.916886,-0.438234,-1.106999,0.412588
9,0.552813,0.717973,-0.174670,1.024134


### Join

SQL style merges. See the [Database style joining](http://pandas.pydata.org/pandas-docs/stable/merging.html#merging-join)

In [79]:
left = pd.DataFrame({'key': ['foo', 'foo'], 'lval': [1, 2]})

In [80]:
right = pd.DataFrame({'key': ['foo', 'foo'], 'rval': [4, 5]})

In [81]:
left

,key,lval
0,foo,1
1,foo,2


In [82]:
right

,key,rval
0,foo,4
1,foo,5


In [83]:
pd.merge(left, right, on='key')

,key,lval,rval
0,foo,1,4
1,foo,1,5
2,foo,2,4
3,foo,2,5


### Append

Append rows to a dataframe. See the [Appending](http://pandas.pydata.org/pandas-docs/stable/merging.html#merging-concatenation)

In [84]:
df = pd.DataFrame(np.random.randn(8, 4), columns=['A','B','C','D'])

In [85]:
df

,A,B,C,D
0,0.693304,-0.669732,-1.411137,-0.139715
1,0.548362,1.863729,-0.846143,-0.030377
2,-1.323827,-0.132981,0.819597,0.566720
3,-0.221026,-1.452072,-1.273640,-0.620831
4,0.565701,0.197159,-1.449701,2.314458
5,-0.754984,0.907898,-0.696003,-0.764968
6,1.563572,0.872210,-0.079432,-0.616406
7,-0.822170,-0.815921,0.105417,0.188397


In [86]:
s = df.iloc[3]

In [87]:
df.append(s, ignore_index=True)

,A,B,C,D
0,0.693304,-0.669732,-1.411137,-0.139715
1,0.548362,1.863729,-0.846143,-0.030377
2,-1.323827,-0.132981,0.819597,0.566720
3,-0.221026,-1.452072,-1.273640,-0.620831
4,0.565701,0.197159,-1.449701,2.314458
5,-0.754984,0.907898,-0.696003,-0.764968
6,1.563572,0.872210,-0.079432,-0.616406
7,-0.822170,-0.815921,0.105417,0.188397
8,-0.221026,-1.452072,-1.273640,-0.620831


## Grouping

By “group by” we are referring to a process involving one or more of the following steps

* **Splitting** the data into groups based on some criteria
* **Applying** a function to each group independently
* **Combining** the results into a data structure

See the [Grouping section](http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby)

In [166]:
pdf = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
                                    'B' : ['one', 'one', 'two', 'three','two', 'two', 'one', 'three'],
                                    'C' : np.random.randn(8),
                                     'D' : np.random.randn(8)})
df = spark.from_pandas(pdf)

In [167]:
df

,A,B,C,D
0,foo,one,0.096617,-1.470267
1,bar,one,0.988536,-0.745742
2,foo,two,0.640741,-0.647182
3,bar,three,2.420534,-1.558839
4,foo,two,0.757523,-1.260813
5,bar,two,0.241265,0.124168
6,foo,one,0.929668,1.433707
7,foo,three,0.075146,-0.572402


Grouping and then applying a function sum to the resulting groups.

In [169]:
# TODO: indexing in groups
df.groupby('A').sum()

,A,sum(__index_level_0__),sum(C),sum(D)
0,bar,9,3.650335,-2.180414
1,foo,19,2.499695,-2.516956


In [170]:
pdf.groupby('A').sum()

,C,D
A,,
bar,3.650335,-2.180414
foo,2.499695,-2.516956


In [171]:
df.groupby(['A','B']).sum()

,A,B,sum(__index_level_0__),sum(C),sum(D)
0,foo,one,6,1.026285,-0.036560
1,foo,two,6,1.398264,-1.907994
2,bar,three,3,2.420534,-1.558839
3,foo,three,7,0.075146,-0.572402
4,bar,two,5,0.241265,0.124168
5,bar,one,1,0.988536,-0.745742


## Reshaping

See the sections on [Hierarchical Indexing](http://pandas.pydata.org/pandas-docs/stable/advanced.html#advanced-hierarchical) and [Reshaping](http://pandas.pydata.org/pandas-docs/stable/reshaping.html#reshaping-stacking).

### Stack

In [92]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
                                ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]))

In [93]:
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])

In [94]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])

In [95]:
df2 = df[:4]

In [96]:
df2

A         B
first second                    
bar   one     0.062197  1.746846
      two     0.085027  0.503990
baz   one    -0.187553 -0.496627
      two    -1.122062  1.027859

The stack() method “compresses” a level in the DataFrame’s columns.

In [97]:
stacked = df2.stack()

In [98]:
stacked

first  second   
bar    one     A    0.062197
               B    1.746846
       two     A    0.085027
               B    0.503990
baz    one     A   -0.187553
               B   -0.496627
       two     A   -1.122062
               B    1.027859
dtype: float64

With a “stacked” DataFrame or Series (having a MultiIndex as the index), the inverse operation of stack() is unstack(), which by default unstacks the **last level**:

In [99]:
stacked.unstack()

A         B
first second                    
bar   one     0.062197  1.746846
      two     0.085027  0.503990
baz   one    -0.187553 -0.496627
      two    -1.122062  1.027859

In [100]:
stacked.unstack(1)

second        one       two
first                      
bar   A  0.062197  0.085027
      B  1.746846  0.503990
baz   A -0.187553 -1.122062
      B -0.496627  1.027859

In [101]:
stacked.unstack(0)

first          bar       baz
second                      
one    A  0.062197 -0.187553
       B  1.746846 -0.496627
two    A  0.085027 -1.122062
       B  0.503990  1.027859

### Pivot Tables

See the section on [Pivot Tables](http://pandas.pydata.org/pandas-docs/stable/reshaping.html#reshaping-pivot).

In [102]:
df = pd.DataFrame({'A' : ['one', 'one', 'two', 'three'] * 3,
                                    'B' : ['A', 'B', 'C'] * 4,
                                    'C' : ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 2,
                                    'D' : np.random.randn(12),
                                    'E' : np.random.randn(12)})

In [103]:
df

,A,B,C,D,E
0,one,A,foo,-1.685998,-0.531468
1,one,B,foo,0.181088,-0.600539
2,two,C,foo,0.569034,0.495495
3,three,A,bar,0.009911,-1.171966
4,one,B,bar,0.715824,0.472303
5,one,C,bar,0.756677,0.869227
6,two,A,foo,0.514357,-1.384513
7,three,B,foo,-0.503018,-0.152671
8,one,C,foo,-1.069953,-1.183500
9,one,A,bar,-0.758206,1.205255


We can produce pivot tables from this data very easily:

In [104]:
pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'])

C             bar       foo
A     B                    
one   A -0.758206 -1.685998
      B  0.715824  0.181088
      C  0.756677 -1.069953
three A  0.009911       NaN
      B       NaN -0.503018
      C  0.647521       NaN
two   A       NaN  0.514357
      B  1.399778       NaN
      C       NaN  0.569034

## Time Series

pandas has simple, powerful, and efficient functionality for performing resampling operations during frequency conversion (e.g., converting secondly data into 5-minutely data). This is extremely common in, but not limited to, financial applications. See the [Time Series section](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#timeseries)

In [105]:
rng = pd.date_range('1/1/2012', periods=100, freq='S')

In [106]:
ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)

In [107]:
ts.resample('5Min').sum()

2012-01-01    23087
Freq: 5T, dtype: int64

Time zone representation

In [108]:
rng = pd.date_range('3/6/2012 00:00', periods=5, freq='D')

In [109]:
ts = pd.Series(np.random.randn(len(rng)), rng)

In [110]:
ts

2012-03-06    0.087342
2012-03-07   -0.386523
2012-03-08   -1.276286
2012-03-09   -0.858647
2012-03-10    1.148225
Freq: D, dtype: float64

In [111]:
ts_utc = ts.tz_localize('UTC')

In [112]:
ts_utc

2012-03-06 00:00:00+00:00    0.087342
2012-03-07 00:00:00+00:00   -0.386523
2012-03-08 00:00:00+00:00   -1.276286
2012-03-09 00:00:00+00:00   -0.858647
2012-03-10 00:00:00+00:00    1.148225
Freq: D, dtype: float64


Convert to another time zone

In [113]:
ts_utc.tz_convert('US/Eastern')

2012-03-05 19:00:00-05:00    0.087342
2012-03-06 19:00:00-05:00   -0.386523
2012-03-07 19:00:00-05:00   -1.276286
2012-03-08 19:00:00-05:00   -0.858647
2012-03-09 19:00:00-05:00    1.148225
Freq: D, dtype: float64


Converting between time span representations

In [114]:
rng = pd.date_range('1/1/2012', periods=5, freq='M')

In [115]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [116]:
ts

2012-01-31   -1.092484
2012-02-29    0.347855
2012-03-31   -1.440464
2012-04-30   -0.390041
2012-05-31    0.017284
Freq: M, dtype: float64

In [117]:
ps = ts.to_period()

In [118]:
ps

2012-01   -1.092484
2012-02    0.347855
2012-03   -1.440464
2012-04   -0.390041
2012-05    0.017284
Freq: M, dtype: float64

In [119]:
ps.to_timestamp()

2012-01-01   -1.092484
2012-02-01    0.347855
2012-03-01   -1.440464
2012-04-01   -0.390041
2012-05-01    0.017284
Freq: MS, dtype: float64

Converting between period and timestamp enables some convenient arithmetic functions to be used. In the following example, we convert a quarterly frequency with year ending in November to 9am of the end of the month following the quarter end:

In [120]:
prng = pd.period_range('1990Q1', '2000Q4', freq='Q-NOV')

In [121]:
ts = pd.Series(np.random.randn(len(prng)), prng)

In [122]:
ts.index = (prng.asfreq('M', 'e') + 1).asfreq('H', 's') + 9

In [123]:
ts.head()

1990-03-01 09:00    0.469320
1990-06-01 09:00    1.020125
1990-09-01 09:00    1.046608
1990-12-01 09:00    0.358405
1991-03-01 09:00   -0.119711
Freq: H, dtype: float64

## Categoricals

Since version 0.15, pandas can include categorical data in a DataFrame. For full docs, see the [categorical introduction](http://pandas.pydata.org/pandas-docs/stable/categorical.html#categorical) and the [API documentation](http://pandas.pydata.org/pandas-docs/stable/api.html#api-categorical).

In [124]:
df = pd.DataFrame({"id":[1,2,3,4,5,6], "raw_grade":['a', 'b', 'b', 'a', 'a', 'e']})

Convert the raw grades to a categorical data type.

In [125]:
df["grade"] = df["raw_grade"].astype("category")

In [126]:
df["grade"]

0    a
1    b
2    b
3    a
4    a
5    e
Name: grade, dtype: category
Categories (3, object): [a, b, e]

Rename the categories to more meaningful names (assigning to Series.cat.categories is inplace!)

In [127]:
df["grade"].cat.categories = ["very good", "good", "very bad"]

Reorder the categories and simultaneously add the missing categories (methods under Series .cat return a new Series per default).

In [128]:
df["grade"] = df["grade"].cat.set_categories(["very bad", "bad", "medium", "good", "very good"])

In [129]:
df["grade"]

0    very good
1         good
2         good
3    very good
4    very good
5     very bad
Name: grade, dtype: category
Categories (5, object): [very bad, bad, medium, good, very good]

Sorting is per order in the categories, not lexical order.

In [130]:
df.sort_values(by="grade")

,id,raw_grade,grade
5,6,e,very bad
1,2,b,good
2,3,b,good
0,1,a,very good
3,4,a,very good
4,5,a,very good


Grouping by a categorical column shows also empty categories.

In [131]:
df.groupby("grade").size()

grade
very bad     1
bad          0
medium       0
good         2
very good    3
dtype: int64

## Plotting
[Plotting](http://pandas.pydata.org/pandas-docs/stable/visualization.html#visualization) docs.

In [132]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))

In [133]:
ts = ts.cumsum()

In [134]:
ts.plot()

ModuleNotFoundError: No module named 'matplotlib'

On DataFrame, plot() is a convenience to plot all of the columns with labels:

In [ ]:
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index,
                  columns=['A', 'B', 'C', 'D'])

In [ ]:
df = df.cumsum()

In [ ]:
plt.figure(); df.plot(); plt.legend(loc='best')

## Getting Data In/Out

### CSV
[Writing to a csv file](http://pandas.pydata.org/pandas-docs/stable/io.html#io-store-in-csv)

In [ ]:
df.to_csv('foo.csv')

[Reading from a csv file](http://pandas.pydata.org/pandas-docs/stable/io.html#io-read-csv-table)

In [ ]:
pd.read_csv('foo.csv')

### HDF5
Reading and writing to [HDFStores](http://pandas.pydata.org/pandas-docs/stable/io.html#io-hdf5)

Writing to a HDF5 Store

In [ ]:
df.to_hdf('foo.h5','df')

Reading from a HDF5 Store

In [ ]:
pd.read_hdf('foo.h5','df')

### Excel

Reading and writing to [MS Excel](http://pandas.pydata.org/pandas-docs/stable/io.html#io-excel)

Writing to an excel file

In [ ]:
df.to_excel('foo.xlsx', sheet_name='Sheet1')

Reading from an excel file

In [ ]:
pd.read_excel('foo.xlsx', 'Sheet1', index_col=None, na_values=['NA'])

### Gotchas
If you are trying an operation and you see an exception like:

In [ ]:
if pd.Series([False, True, False]):
    print("I was true")

See [Comparisons](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-compare) for an explanation and what to do.

See [Gotchas](http://pandas.pydata.org/pandas-docs/stable/gotchas.html#gotchas) as well.